# 텍스트 클렌징 테스트

In [1]:
import requests
from bs4 import BeautifulSoup

import re
import textacy.preprocessing as tprep
import nltk
import spacy

from konlpy.tag import Okt

import numpy as np
import pandas as pd

import datetime
import time

from tqdm import tqdm

import pickle

### raw data 수집 함수: 오늘 첫 10장 분량만 획득

In [2]:
def crawl(start=None, end=None) :
    BASE_URL = 'https://news.naver.com/main/list.naver?mode=LSD&mid=sec&listType=title&'

    page = 1
    date = datetime.datetime.now()
    titles = []

    for tiem in tqdm(range(50)):
        url = BASE_URL
        url += f'date={str(date.year) + str(date.month).zfill(2) + str(date.day).zfill(2)}&'
        url += f'page={page}'
        
        res = requests.get(url)
        bs = BeautifulSoup(res.text)
        titles.extend([e.get_text() for e in bs.find_all('a', class_="nclicks(fls.list)")])

        page += 1
        time.sleep(1)

    return titles

In [3]:
try:
    with open('raw_titles.pkl', 'rb') as f:
        raw_titles = pickle.load(f)

except:
    raw_titles = crawl()

In [4]:
np.random.choice(raw_titles, 10)

array(["'클럽 마약' 허벅지에 숨겨 밀수입한 운반책들 2심도 실형", "'야구명가' 컴투스, 야구 인기 급등에 신바람 탔다",
       '가자전쟁 6개월…“아동 7000명 심각한 영양실조”',
       "용인시, 보정동 카페거리 등 5곳 '골목형 상점가' 지정 추진", '‘당신을 위한 튜브’… 흥미진진한 알레고리',
       '비욘세, 흑인여성 첫 ‘컨트리’ 1위…귀를 의심한 충격적 걸작',
       "'북중친선의 해' 개막식에서 연설하는 중국 자오러지",
       "[영상] 탐사보고서 기록 '주문하신 커피 나왔습니다' 예고편",
       '미국·필리핀, 외교·국방·안보보좌관 회의…“중국 공세 대응 공조”',
       '엔클로니, 2023년 설립 이래 최대 매출…IPO 청신호'], dtype='<U98')

with open('raw_titles.pkl', 'wb') as f:
    pickle.dump(raw_titles, f)

pat = r'[\(\{\[]+[ㄱ-ㅎ-가-힣\w\s,]+[^ㄱ-ㅎ-가-힣\w\s]*[\]\}\)]+'

https://n.news.naver.com/mnews/article/001/0014628755?rc=N&ntype=RANKING 이건 뭐지?

clean_titles = list(map(normalize_punct, raw_titles))
clean_titles

[연합뉴스 이 시각 헤드라인] - 10:30
인천∼제주
고대장이 간다? 유이도도 한다!
AI=반도체+구리?
SLOVENIA-KAMNIK-TULIPS
3.2%
딴 남자 만나지? 
여성ㆍ퀴어
19∼21일
″가자지구 중부 난민캠프 공격..언론인들 부상
Roundup:
주유소 기름값 오름세 지속‥, 찾길‥제가
K-관광

tprep.normalize.quotation_marks('″가자지구 중부 난민캠프 공격..언론인들 부상')

In [5]:
ord('″')

8243

In [6]:
hex(8243)

'0x2033'

normalize_punct('가자지구 중부 난민캠프 공격..언론인들 부상')

In [7]:
import unicodedata

unicodedata.name('″')

'DOUBLE PRIME'

In [8]:
tprep.normalize.unicode('″')

'″'

In [9]:
tprep.remove.punctuation('″가자지구 중부 난민캠프 공격..언론인들 부상')

' 가자지구 중부 난민캠프 공격  언론인들 부상'

## 클렌징 품질 향상: 문장 부호 클렌징 점수를 기준으로 평가

{\text{문장부호}+\text{[속보], (단독)식의 단어들 등}} \over \text{문장 길이}}

In [10]:
"""
    클렌징 점수 평가 방법
        1. 전체 데이터셋을 검사해 문장 부호 집합 생성
        2. 각 제목으로부터 문장 부호 + (~) / 단어 구분
        3. 각 제목에 있는 문장 부호, (~) 존재 여부, 1.23% 따위의 표현 존재 여부, 불릿 표시 존재 여부 등을 OHV로 변환

        결과: 문장부호 정규화시킨 제목 -> 문장부호 없어진 제목
"""

'\n    클렌징 점수 평가 방법\n        1. 전체 데이터셋을 검사해 문장 부호 집합 생성\n        2. 각 제목으로부터 문장 부호 + (~) / 단어 구분\n        3. 각 제목에 있는 문장 부호, (~) 존재 여부, 1.23% 따위의 표현 존재 여부, 불릿 표시 존재 여부 등을 OHV로 변환\n\n        결과: 문장부호 정규화시킨 제목 -> 문장부호 없어진 제목\n'

크롤링
-> 문장부호 정규화
-> 불필요한 문장부호 + 표현 제거

In [11]:
def get_punct_list(title):
    return re.findall(r'[^ㄱ-ㅎ-가-힣\w\s\(\{\[\)\}\]]', title)

In [12]:
def get_punct_set(titles):
    punct_set = set()
    titles.apply(lambda x: punct_set.update(get_punct_list(x)))
    return punct_set

In [13]:
titles = pd.Series(raw_titles)
punct_set = get_punct_set(titles)
punct_set

{'!',
 '"',
 '#',
 '%',
 '&',
 "'",
 '+',
 ',',
 '.',
 '/',
 ':',
 '=',
 '>',
 '?',
 '`',
 '~',
 '·',
 '‘',
 '’',
 '“',
 '”',
 '‥',
 '…',
 '‧',
 '″',
 '℃',
 '↑',
 '→',
 '↓',
 '∼',
 '㈜',
 '㎏',
 '㎝',
 '㎞',
 '㎡',
 '％',
 '［',
 '］'}

- 빈 집합 생성
- 각 문장에 있는 문장 부호 리스트 획득
- 리스트 원소를 집합의 요소로 추가
- 최종 결과물: 데이터셋 내에 있는 문장 부호 리스트

In [14]:
ord('［')

65339

In [15]:
ord('[')

91

In [16]:
print(*[(p, ord(p)) for p in punct_set])

('“', 8220) ('?', 63) ('”', 8221) ('·', 183) ('∼', 8764) ('+', 43) (':', 58) ('‥', 8229) ('>', 62) ('"', 34) ('.', 46) ('/', 47) ('=', 61) ('~', 126) ('‘', 8216) (',', 44) ('’', 8217) ('#', 35) ('］', 65341) ("'", 39) ('［', 65339) ('‧', 8231) ('℃', 8451) ('…', 8230) ('″', 8243) ('％', 65285) ('㎡', 13217) ('↓', 8595) ('&', 38) ('㎝', 13213) ('↑', 8593) ('→', 8594) ('!', 33) ('%', 37) ('㎞', 13214) ('㎏', 13199) ('㈜', 12828) ('`', 96)


In [17]:
'민주 “국정 쇄신은 ‘해병대원 순직 수사외압’ 특검 수용서 시작”'.translate({ord('“'): ord('"')})

'민주 "국정 쇄신은 ‘해병대원 순직 수사외압’ 특검 수용서 시작”'

계속 긁어오다 마지막 원소가 직전과 같다면 크롤링 종료? X<br>
<br>
'다음' 버튼이 나와있지 않고 + 페이지 목록에서 마지막 번호면 -> 그 날의 끝으로 간주

아주 적은 빈도의 특수문자들은 치환하지 않고 없애버려서 테이블 크기를 줄이자!
1. 전체 데이터에 있는 특수문자들 목록 + 빈도를 알아낸다
2. 거의 없는 특수문자들이 뭔지 알아낸다
3. 정규화하는 기능을 만들어낼 때<br>
    3-1. 자주 나오는 특수문자들은 치환하고<br>
    3-2. 자주 나오지 않는 특수문자들은 치환하지 않고 제거한다?

In [18]:
(400*50*5+200*50*2)*2.5*1.5/3600

125.0

In [19]:
1000**3

1000000000

1. 한 달 분량 기사 긁어오기
2. 특수문자 빈도 알아내기
3. 하위 n개 / n% 특수문자 알아내기
4. 정규화를 위한 테이블 만들 때
    - 하위 n개 / n% 특수문자는 테이블로 만들지 않음
    - 그 외엔 테이블 만들기
5. 4에서 만든 테이블로 특수문자 정규화
6. 정규화된 특수문자 적절하게 처리 + 테이블 목록에 없는 특수문자 (즉 하위 n개 / n% 특수문자) 제거

- 처리 전 불순도 측정 (기준: 특수문자 수 + (~) 수 이용)
- 특수문자 빈도 알아내기
- 하위 n개 / n% 특수문자 알아내기
- 정규화를 위한 테이블 구축 : 이 때
    - 하위 n개 / n% 특수문자는 테이블로 만들지 않음 -> 바로 제거해버릴 것들
    - 그 외엔 테이블 만들기 -> 정규화할 것들
- 특수문자 정규화
- 불필요한 특수문자 제거 (정규화된 특수문자 + 하위 특수문자)
- (~) 따위의 표현 제거
- 처리 후 불순도 측정
- 만족할만한 결과가 나오면 토큰화 + 언어적 처리(ngram, 조사 제거 등) 진행

In [20]:
def get_punct_freq(titles):
    punct_set = get_punct_set(titles)
    punct_freq = {p : 0 for p in punct_set}
    for t in titles:
        for p in get_punct_list(t):
            punct_freq[p] += 1

    return punct_freq

In [21]:
punct_freq = get_punct_freq(titles)
punct_freq

{'“': 220,
 '?': 165,
 '”': 217,
 '·': 371,
 '∼': 11,
 '+': 10,
 ':': 7,
 '‥': 22,
 '>': 7,
 '"': 933,
 '.': 486,
 '/': 1,
 '=': 1,
 '~': 20,
 '‘': 256,
 ',': 783,
 '’': 260,
 '#': 4,
 '］': 2,
 "'": 1080,
 '［': 2,
 '‧': 1,
 '℃': 2,
 '…': 1004,
 '″': 1,
 '％': 2,
 '㎡': 8,
 '↓': 9,
 '&': 8,
 '㎝': 1,
 '↑': 9,
 '→': 11,
 '!': 21,
 '%': 77,
 '㎞': 2,
 '㎏': 1,
 '㈜': 1,
 '`': 10}

In [22]:
punct_info = pd.DataFrame([punct_freq.keys(), [ord(p) for p in punct_freq.keys()], punct_freq.values()]).T
punct_info.sort_values(ascending=False, by=2)

,0,1,2
19,',39,1080
23,…,8230,1004
9,"""",34,933
15,",",44,783
10,.,46,486
3,·,183,371
16,’,8217,260
14,‘,8216,256
0,“,8220,220
2,”,8221,217


In [23]:
len(punct_set)

38

In [24]:
def get_impurity_score(title:str):
    cpy = title[:]
    cpy = re.sub(r'[\(\{\[]+[ㄱ-ㅎ-가-힣\w\s,]+[^ㄱ-ㅎ-가-힣\w\s]*[\]\}\)]+', '.', cpy)
    cpy = re.sub(r'\s', '', cpy)

    n_chars = len(cpy) if len(cpy) != 0 else 1 # (copyright) 같은 제목 때문에 0 발생 -> 1로 처리
    n_puncts = len(get_punct_list(cpy))
    
    return round(n_puncts / n_chars, 3)

In [25]:
titles[0], get_impurity_score(titles[0])

('김포시, 한강중앙공원 물놀이장 탈의실 설치 완료', 0.048)

In [26]:
titles.apply(get_impurity_score).mean()

0.10291999999999998

In [27]:
titles.apply(get_impurity_score).sort_values(ascending=False)

842     1.000
1547    1.000
162     1.000
841     1.000
2309    0.522
        ...  
1346    0.000
1347    0.000
1351    0.000
1352    0.000
2499    0.000
Length: 2500, dtype: float64

In [39]:
top_10_impurities = titles.apply(get_impurity_score).sort_values(ascending=False).head(10).index
titles[top_10_impurities]

842                            [전국 주요 신문 톱뉴스](13일 조간)
1547                                   [김회룡의 시사 TOON]
162                                       (Copyright)
841                                [주요 신문 사설](13일 조간)
2309                     [노컷한컷]175+12+3+1+1>>>>>>>108
2388                          [오늘의 주요일정]사회(4월13일 토요일)
2298                               [K-소비자 브랜드 대상] ㈜애반
27                   與, 새 지도체제 '이견'..."비대위" vs "전당대회"
1059                 "벌써 여름?"…서울·춘천 낮 최고 '30도' [내일날씨]
1947    39.09%→54.49%→54.75%→58.3%…선거 치를수록 '우상향' 비결은?
dtype: object

In [29]:
TRANSLATE_TABLE = { # 치환 후 없앨 것들 목록
    ord(x) : ord(y) 
    for x, y
    in [
        ('［', '['),
        ('％', '%'),
        ('］', ']'),
        ('″', '"'),
        ('”', '"'),
        ('‘', "'"),
        ('∼', '~'),
        ('`', "'"),
        ('’', "'"),
        ('“', '"')
    ]
}

TRANSLATE_TABLE

{65339: 91,
 65285: 37,
 65341: 93,
 8243: 34,
 8221: 34,
 8216: 39,
 8764: 126,
 96: 39,
 8217: 39,
 8220: 34}

In [30]:
def rep_sokbo_into_ub(title):
    return re.sub(r'[\(\{\[]+[ㄱ-ㅎ-가-힣\w\s,]+[^ㄱ-ㅎ-가-힣\w\s]*[\]\}\)]+', '_', title)

In [31]:
def normalize_punct(title):
    title = rep_sokbo_into_ub(title) # (속보), [단독] 따위의 [000의 건강상식]과 같은 요소들은 .으로 변경
    title = title.translate(TRANSLATE_TABLE)
    
    title = tprep.normalize.quotation_marks(title) # 따옴표 정규화
    
    title = re.sub(r'\.\.(\.)?', '…', title) # 말줄임표 정규화 ('..' , '...' -> '…')
    
    title = tprep.normalize.bullet_points(title)
    title = re.sub(r'·', ' ', title) # 불릿 표현 정규화 + 띄어쓰기로 변형 -> 추후 품사 태깅 등을 통해 낱말 조합 등 진행

    # 필요 없는 문장부호 제거 
    # + '‥' 추가 : 041324 1318
    title = tprep.remove.punctuation(title, only=['\'', '\"', '…', ',', '‥', '!', '@', '#', '&', '/', '+', '=', '~', '?', '>', '_', '㈜'])
    
    title = re.sub('\s+', ' ', title) # 위에서 생긴 연속 공백 제거
    title = title.strip() # 양 끝 공백 제거
    
    return title

In [32]:
clean_titles = titles.apply(normalize_punct)
clean_titles.apply(get_impurity_score).mean()

0.0035619999999999996

In [33]:
clean_titles[top_10_impurities]

842                                               
1547                                              
162                                               
841                                               
2309                              175 12 3 1 1 108
2388                                            사회
2298                                            애반
27                         與 새 지도체제 이견 비대위 vs 전당대회
1059                          벌써 여름 서울 춘천 낮 최고 30도
1947    39.09%→54.49%→54.75%→58.3% 선거 치를수록 우상향 비결은
dtype: object

In [34]:
repr(clean_titles[842])

"''"

In [35]:
len(clean_titles[842])

0

In [36]:
titles.info(memory_usage='deep')

<class 'pandas.core.series.Series'>
RangeIndex: 2500 entries, 0 to 2499
Series name: None
Non-Null Count  Dtype 
--------------  ----- 
2500 non-null   object
dtypes: object(1)
memory usage: 330.5 KB


In [37]:
clean_titles.info(memory_usage='deep')

<class 'pandas.core.series.Series'>
RangeIndex: 2500 entries, 0 to 2499
Series name: None
Non-Null Count  Dtype 
--------------  ----- 
2500 non-null   object
dtypes: object(1)
memory usage: 314.5 KB


In [38]:
(clean_titles == '').sum()

4